In [1]:
import pandas as pd   
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score, mean_squared_error, accuracy_score, precision_score, recall_score, f1_score

from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
df_eurchf = pd.read_csv('/content/EUR-CHF Exchange Rate.csv')
df_trends = pd.read_excel('/content/World_GT_Full.xlsx')

In [3]:
df_trends['Date'] = pd.to_datetime(df_trends.iloc[:, 0], infer_datetime_format=True)
df_eurchf['Date'] = pd.to_datetime(df_eurchf.iloc[:, 0], infer_datetime_format=True)

In [4]:
df_eurchf['EUR/CHF Log Difference'] = np.log(df_eurchf[' value']) - (np.log(df_eurchf[' value']).shift(1))
df_eurchf['CHF/EUR Log Difference'] = -1 * df_eurchf['EUR/CHF Log Difference']
df_eurchf = df_eurchf.dropna()

In [5]:
ts_df = df_eurchf.merge(df_trends, how = 'inner', on = ['Date'])

In [6]:
ts_df = ts_df.drop(['log_difference',' value'], axis = 1)
ts_df = ts_df.set_index(ts_df.iloc[:,0])
ts_df = ts_df.drop(['Date', 'EUR/CHF Log Difference'], axis = 1)
ts_df

,CHF/EUR Log Difference,swiss franc,Europe,EUR,Switzerland,CHF,NASDAQ,S&P500,VIX,volatility,Germany,UK,sterling pound,GBP,Greece,Italy,Spain,France,DAX,unemployment,recession,euro,crisis,fed,finance,financial crisis,federal reserve,gold,inflation,treasuries,OPEC,contagion,corruption,disaster,futures,forwards,president,parliament,aid,EU,...,steel,silver,bronze,corn,commodities,natural gas,agriculture,emerging market,trade,interest rate,GDP,exports,imports,jobs,tax,tariff,uncertainty,gini,inequality,insurance,pension,liquidity,diversification,alpha,currency,CAPM,fixed income,commission,SEC,yield,real estate,eurodollar,regulations,deficit,yield curve,LIBOR,2008,bubble,energy,1939
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01-02,0.000898,0,58,12,8,2,22,0,19,31,10,76,0,1,15,5,23,32,3,6,0,5,12,16,47,0,12,18,32,0,8,0,6,5,12,8,3,1,23,3,...,18,36,20,10,26,27,27,0,38,18,21,33,90,11,21,34,15,0,0,68,46,0,8,51,48,0,0,13,27,41,68,0,33,15,0,23,0,30,43,28
2004-01-05,-0.002498,0,39,0,4,0,14,0,30,0,7,23,0,0,14,6,17,15,0,7,0,3,26,10,41,0,0,27,20,0,0,0,0,15,3,0,5,0,10,2,...,15,42,39,5,14,30,56,0,40,37,100,14,28,16,21,23,0,14,59,50,19,0,0,55,66,0,0,21,68,30,82,0,34,51,0,0,0,23,52,15
2004-01-06,-0.002875,0,73,5,6,2,35,0,10,30,12,100,0,0,23,6,24,40,2,10,0,6,20,18,87,0,27,19,64,0,3,0,41,8,19,0,5,8,20,5,...,29,37,43,21,25,19,42,12,60,19,31,36,41,14,23,59,10,5,17,100,67,15,11,57,74,22,39,26,70,31,54,15,80,47,74,75,0,29,84,45
2004-01-07,0.000064,0,60,2,7,4,32,0,0,32,10,79,0,0,22,7,31,31,1,8,1,6,26,32,71,0,28,20,41,0,6,0,9,11,9,8,5,4,18,4,...,27,39,33,12,18,48,60,9,62,17,49,34,50,14,25,34,27,4,19,76,34,29,0,48,75,10,58,24,46,23,60,11,58,37,40,25,0,31,87,20
2004-01-08,0.001341,0,67,7,8,2,48,0,6,11,9,82,8,0,17,7,34,35,4,5,1,5,19,24,63,2,23,18,41,0,4,0,4,11,21,23,5,6,21,5,...,29,35,38,15,23,40,49,0,70,23,41,43,68,12,22,50,30,6,44,81,52,18,0,43,74,0,38,21,33,28,55,0,79,55,0,51,0,30,80,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-09,0.005245,1,23,14,8,13,45,31,16,9,6,38,2,6,13,4,9,12,22,13,4,7,16,20,63,4,6,27,22,2,2,1,25,18,18,3,3,5,17,8,...,25,36,37,22,40,29,23,2,34,18,15,25,9,9,18,21,7,8,16,39,52,12,1,38,24,8,4,7,44,41,15,1,15,55,2,2,1,37,66,8
2021-09-10,-0.000369,1,20,13,7,12,41,27,10,7,5,37,1,5,11,4,9,11,18,11,3,7,14,19,57,2,6,27,22,2,2,1,19,17,15,2,3,5,15,8,...,23,36,37,23,47,25,20,3,36,16,15,23,7,8,16,18,8,7,14,36,48,11,1,36,22,5,4,6,41,37,14,0,11,45,0,2,1,39,60,7
2021-09-11,-0.000369,2,17,10,7,9,12,22,7,5,4,37,2,4,11,4,8,11,7,6,3,6,12,13,35,3,3,31,17,3,2,2,19,13,4,2,3,4,13,8,...,25,39,36,27,23,19,16,2,59,16,12,15,5,6,12,13,6,6,8,25,42,10,1,31,20,2,2,4,56,27,13,0,9,35,1,0,1,44,44,8


In [7]:
#ts_df.iloc[:,1:155] = ts_df.iloc[:,1:155].diff()

In [8]:
#ts_df = ts_df.iloc[1:,:]

In [9]:
#ts_df.head()

In [10]:
# Creating a copy of the initial datagrame to make various transformations 
data = pd.DataFrame(ts_df.copy())

In [11]:
window_size = 2
data['CHF/EUR_RealVol'] = data['CHF/EUR Log Difference'].rolling(window_size).std()*(252**0.5)

In [12]:
data['CHF/EUR_RealVol'].quantile(0.75) #75th Percentile

0.04887169872649703

In [13]:
# Adding the lag of the target variable from 6 steps back up to 24

for column in data:

  for i in range(1, 14):
      data[f"{column}_lag_{i}"] = data[column].shift(i)

In [14]:
data['HighVolMoment_CHF/EUR'] = [1 if i > 0.05 else 0 for i in data['CHF/EUR_RealVol']]

In [15]:
data

,CHF/EUR Log Difference,swiss franc,Europe,EUR,Switzerland,CHF,NASDAQ,S&P500,VIX,volatility,Germany,UK,sterling pound,GBP,Greece,Italy,Spain,France,DAX,unemployment,recession,euro,crisis,fed,finance,financial crisis,federal reserve,gold,inflation,treasuries,OPEC,contagion,corruption,disaster,futures,forwards,president,parliament,aid,EU,...,energy_lag_1,energy_lag_2,energy_lag_3,energy_lag_4,energy_lag_5,energy_lag_6,energy_lag_7,energy_lag_8,energy_lag_9,energy_lag_10,energy_lag_11,energy_lag_12,energy_lag_13,1939_lag_1,1939_lag_2,1939_lag_3,1939_lag_4,1939_lag_5,1939_lag_6,1939_lag_7,1939_lag_8,1939_lag_9,1939_lag_10,1939_lag_11,1939_lag_12,1939_lag_13,CHF/EUR_RealVol_lag_1,CHF/EUR_RealVol_lag_2,CHF/EUR_RealVol_lag_3,CHF/EUR_RealVol_lag_4,CHF/EUR_RealVol_lag_5,CHF/EUR_RealVol_lag_6,CHF/EUR_RealVol_lag_7,CHF/EUR_RealVol_lag_8,CHF/EUR_RealVol_lag_9,CHF/EUR_RealVol_lag_10,CHF/EUR_RealVol_lag_11,CHF/EUR_RealVol_lag_12,CHF/EUR_RealVol_lag_13,HighVolMoment_CHF/EUR
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01-02,0.000898,0,58,12,8,2,22,0,19,31,10,76,0,1,15,5,23,32,3,6,0,5,12,16,47,0,12,18,32,0,8,0,6,5,12,8,3,1,23,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2004-01-05,-0.002498,0,39,0,4,0,14,0,30,0,7,23,0,0,14,6,17,15,0,7,0,3,26,10,41,0,0,27,20,0,0,0,0,15,3,0,5,0,10,2,...,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2004-01-06,-0.002875,0,73,5,6,2,35,0,10,30,12,100,0,0,23,6,24,40,2,10,0,6,20,18,87,0,27,19,64,0,3,0,41,8,19,0,5,8,20,5,...,52.0,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.038119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2004-01-07,0.000064,0,60,2,7,4,32,0,0,32,10,79,0,0,22,7,31,31,1,8,1,6,26,32,71,0,28,20,41,0,6,0,9,11,9,8,5,4,18,4,...,84.0,52.0,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.0,15.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004228,0.038119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2004-01-08,0.001341,0,67,7,8,2,48,0,6,11,9,82,8,0,17,7,34,35,4,5,1,5,19,24,63,2,23,18,41,0,4,0,4,11,21,23,5,6,21,5,...,87.0,84.0,52.0,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,45.0,15.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.032987,0.004228,0.038119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-09,0.005245,1,23,14,8,13,45,31,16,9,6,38,2,6,13,4,9,12,22,13,4,7,16,20,63,4,6,27,22,2,2,1,25,18,18,3,3,5,17,8,...,70.0,66.0,52.0,46.0,62.0,63.0,64.0,62.0,58.0,43.0,57.0,58.0,62.0,9.0,8.0,8.0,8.0,7.0,8.0,13.0,7.0,5.0,6.0,7.0,6.0,7.0,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0.016606,0.007286,1
2021-09-10,-0.000369,1,20,13,7,12,41,27,10,7,5,37,1,5,11,4,9,11,18,11,3,7,14,19,57,2,6,27,22,2,2,1,19,17,15,2,3,5,15,8,...,66.0,70.0,66.0,52.0,46.0,62.0,63.0,64.0,62.0,58.0,43.0,57.0,58.0,8.0,9.0,8.0,8.0,8.0,7.0,8.0,13.0,7.0,5.0,6.0,7.0,6.0,0.064033,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0.016606,1
2021-09-11,-0.000369,2,17,10,7,9,12,22,7,5,4,37,2,4,11,4,8,11,7,6,3,6,12,13,35,3,3,31,17,3,2,2,19,13,4,2,3,4,13,8,...,60.0,66.0,70.0,66.0,52.0,46.0,62.0,63.0,64.0,62.0,58.0,43.0,57.0,7.0,8.0,9.0,8.0,8.0,8.0,7.0,8.0,13.0,7.0,5.0,6.0,7.0,0.063023,0.064033,0.017547,0.010299,0.008264,0.012409,0.024822,0.019738,0.048743,0.083216,0.072849,0.037503,0.079181,0


In [16]:
data = data.iloc[14:,:]

In [17]:
fin_data = [data.iloc[:,0:1], data.iloc[:,156:]]
final_data = pd.concat(fin_data, axis=1)

In [18]:
final_data

,CHF/EUR Log Difference,CHF/EUR Log Difference_lag_1,CHF/EUR Log Difference_lag_2,CHF/EUR Log Difference_lag_3,CHF/EUR Log Difference_lag_4,CHF/EUR Log Difference_lag_5,CHF/EUR Log Difference_lag_6,CHF/EUR Log Difference_lag_7,CHF/EUR Log Difference_lag_8,CHF/EUR Log Difference_lag_9,CHF/EUR Log Difference_lag_10,CHF/EUR Log Difference_lag_11,CHF/EUR Log Difference_lag_12,CHF/EUR Log Difference_lag_13,swiss franc_lag_1,swiss franc_lag_2,swiss franc_lag_3,swiss franc_lag_4,swiss franc_lag_5,swiss franc_lag_6,swiss franc_lag_7,swiss franc_lag_8,swiss franc_lag_9,swiss franc_lag_10,swiss franc_lag_11,swiss franc_lag_12,swiss franc_lag_13,Europe_lag_1,Europe_lag_2,Europe_lag_3,Europe_lag_4,Europe_lag_5,Europe_lag_6,Europe_lag_7,Europe_lag_8,Europe_lag_9,Europe_lag_10,Europe_lag_11,Europe_lag_12,Europe_lag_13,...,energy_lag_1,energy_lag_2,energy_lag_3,energy_lag_4,energy_lag_5,energy_lag_6,energy_lag_7,energy_lag_8,energy_lag_9,energy_lag_10,energy_lag_11,energy_lag_12,energy_lag_13,1939_lag_1,1939_lag_2,1939_lag_3,1939_lag_4,1939_lag_5,1939_lag_6,1939_lag_7,1939_lag_8,1939_lag_9,1939_lag_10,1939_lag_11,1939_lag_12,1939_lag_13,CHF/EUR_RealVol_lag_1,CHF/EUR_RealVol_lag_2,CHF/EUR_RealVol_lag_3,CHF/EUR_RealVol_lag_4,CHF/EUR_RealVol_lag_5,CHF/EUR_RealVol_lag_6,CHF/EUR_RealVol_lag_7,CHF/EUR_RealVol_lag_8,CHF/EUR_RealVol_lag_9,CHF/EUR_RealVol_lag_10,CHF/EUR_RealVol_lag_11,CHF/EUR_RealVol_lag_12,CHF/EUR_RealVol_lag_13,HighVolMoment_CHF/EUR
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01-23,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,-0.002875,-0.002498,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,73.0,39.0,...,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,84.0,52.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,45.0,15.0,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0.004228,0.038119,0
2004-01-26,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,-0.002875,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,73.0,...,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,84.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,45.0,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0.004228,0
2004-01-27,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,...,73.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0
2004-01-28,0.000383,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,12.0,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,79.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,...,89.0,73.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,9.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,0.019360,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0
2004-01-29,0.002685,0.000383,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.0,12.0,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,72.0,79.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,...,78.0,89.0,73.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,21.0,9.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,0.005731,0.0

#Classification

In [19]:
# Creating a copy of the initial datagrame to make various transformations 
classification_df = pd.DataFrame(final_data.copy())

In [20]:
classification_df

,CHF/EUR Log Difference,CHF/EUR Log Difference_lag_1,CHF/EUR Log Difference_lag_2,CHF/EUR Log Difference_lag_3,CHF/EUR Log Difference_lag_4,CHF/EUR Log Difference_lag_5,CHF/EUR Log Difference_lag_6,CHF/EUR Log Difference_lag_7,CHF/EUR Log Difference_lag_8,CHF/EUR Log Difference_lag_9,CHF/EUR Log Difference_lag_10,CHF/EUR Log Difference_lag_11,CHF/EUR Log Difference_lag_12,CHF/EUR Log Difference_lag_13,swiss franc_lag_1,swiss franc_lag_2,swiss franc_lag_3,swiss franc_lag_4,swiss franc_lag_5,swiss franc_lag_6,swiss franc_lag_7,swiss franc_lag_8,swiss franc_lag_9,swiss franc_lag_10,swiss franc_lag_11,swiss franc_lag_12,swiss franc_lag_13,Europe_lag_1,Europe_lag_2,Europe_lag_3,Europe_lag_4,Europe_lag_5,Europe_lag_6,Europe_lag_7,Europe_lag_8,Europe_lag_9,Europe_lag_10,Europe_lag_11,Europe_lag_12,Europe_lag_13,...,energy_lag_1,energy_lag_2,energy_lag_3,energy_lag_4,energy_lag_5,energy_lag_6,energy_lag_7,energy_lag_8,energy_lag_9,energy_lag_10,energy_lag_11,energy_lag_12,energy_lag_13,1939_lag_1,1939_lag_2,1939_lag_3,1939_lag_4,1939_lag_5,1939_lag_6,1939_lag_7,1939_lag_8,1939_lag_9,1939_lag_10,1939_lag_11,1939_lag_12,1939_lag_13,CHF/EUR_RealVol_lag_1,CHF/EUR_RealVol_lag_2,CHF/EUR_RealVol_lag_3,CHF/EUR_RealVol_lag_4,CHF/EUR_RealVol_lag_5,CHF/EUR_RealVol_lag_6,CHF/EUR_RealVol_lag_7,CHF/EUR_RealVol_lag_8,CHF/EUR_RealVol_lag_9,CHF/EUR_RealVol_lag_10,CHF/EUR_RealVol_lag_11,CHF/EUR_RealVol_lag_12,CHF/EUR_RealVol_lag_13,HighVolMoment_CHF/EUR
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01-23,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,-0.002875,-0.002498,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,73.0,39.0,...,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,84.0,52.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,45.0,15.0,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0.004228,0.038119,0
2004-01-26,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,-0.002875,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,73.0,...,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,84.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,45.0,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0.004228,0
2004-01-27,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,...,73.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0
2004-01-28,0.000383,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,12.0,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,79.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,...,89.0,73.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,9.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,0.019360,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0
2004-01-29,0.002685,0.000383,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.0,12.0,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,72.0,79.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,...,78.0,89.0,73.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,21.0,9.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,0.005731,0.0

In [21]:
y = classification_df['HighVolMoment_CHF/EUR']
X = classification_df.drop(['CHF/EUR Log Difference', 'HighVolMoment_CHF/EUR'], axis=1)

#Train-Test Split 

In [22]:
# for time-series cross-validation set 5 folds 
tscv = TimeSeriesSplit(n_splits=5)

In [23]:
def timeseries_train_test_split(X, y, test_size):
    """
        Perform train-test split with respect to time series structure
    """
    
    # get the index after which test set starts
    test_index = int(len(X)*(1-test_size))
    
    X_train = X.iloc[:test_index]
    y_train = y.iloc[:test_index]
    X_test = X.iloc[test_index:]
    y_test = y.iloc[test_index:]
    
    return X_train, X_test, y_train, y_test

In [24]:
# reserve 30% of data for testing
X_train, X_test, y_train, y_test = timeseries_train_test_split(X, y, test_size=0.3)

#Testing Models

##Benchmark

##Model 1: Logistic Regression


In [25]:
from sklearn.linear_model import LogisticRegression
LR1 = LogisticRegression(random_state=42, max_iter=10000).fit(X_train, y_train)

In [26]:
prob_predictions1 = LR1.predict_proba(X_test)
pred1 = prob_predictions1[:,1]

In [27]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="LR Predicted",
    mode="markers+lines", x=X_test.index, y=pred1
))

fig1.show()

In [28]:
prediction1 = LR1.predict(X_test)

In [29]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction1)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction1)}")
print(f"Precision Score: {precision_score(y_test, prediction1)}")
print(f"F1 Score: {f1_score(y_test, prediction1)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred1, squared=False)}")

Recall Score: 0.17204301075268819
Accuracy Score: 0.7201413427561838
Precision Score: 0.22535211267605634
F1 Score: 0.1951219512195122
RMSE of Probability Score: 0.5190424082863306


##Model 2: SGD Classifier

In [30]:
from sklearn.linear_model import SGDClassifier
from sklearn.calibration import CalibratedClassifierCV
SGD2 = CalibratedClassifierCV(SGDClassifier(random_state=42, max_iter=10000)).fit(X_train, y_train)

In [31]:
prob_predictions2 = SGD2.predict_proba(X_test)
pred2 = prob_predictions2[:,1]

In [32]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="SGD2 Predicted",
    mode="markers+lines", x=X_test.index, y=pred2
))

fig1.show()

In [33]:
prediction2 = SGD2.predict(X_test)

In [34]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction2)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction2)}")
print(f"Precision Score: {precision_score(y_test, prediction2)}")
print(f"F1 Score: {f1_score(y_test, prediction2)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred2, squared=False)}")

Recall Score: 0.0
Accuracy Score: 0.8028268551236749
Precision Score: 0.0
F1 Score: 0.0
RMSE of Probability Score: 0.4032105807452512


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



##Model 3: Passive Aggressive Classifier 

In [35]:
from sklearn.linear_model import PassiveAggressiveClassifier
PAC3 = CalibratedClassifierCV(PassiveAggressiveClassifier(max_iter=10000, random_state=42)).fit(X_train, y_train)

In [36]:
prob_predictions3 = PAC3.predict_proba(X_test)
pred3 = prob_predictions3[:,1]

In [37]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="PAC3 Predicted",
    mode="markers+lines", x=X_test.index, y=pred3
))

fig1.show()

In [38]:
prediction3 = PAC3.predict(X_test)

In [39]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction3)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction3)}")
print(f"Precision Score: {precision_score(y_test, prediction3)}")
print(f"F1 Score: {f1_score(y_test, prediction3)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred3, squared=False)}")

Recall Score: 0.017921146953405017
Accuracy Score: 0.7901060070671378
Precision Score: 0.17857142857142858
F1 Score: 0.032573289902280124
RMSE of Probability Score: 0.44171992091609563


##Model 4: Perceptron

In [40]:
from sklearn.linear_model import Perceptron
Per4 = CalibratedClassifierCV(Perceptron(tol=1e-3, random_state=42)).fit(X_train, y_train)

In [41]:
prob_predictions4 = Per4.predict_proba(X_test)
pred4 = prob_predictions4[:,1]

In [42]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="Per4 Predicted",
    mode="markers+lines", x=X_test.index, y=pred4
))

fig1.show()

In [43]:
prediction4 = Per4.predict(X_test)

In [44]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction4)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction4)}")
print(f"Precision Score: {precision_score(y_test, prediction4)}")
print(f"F1 Score: {f1_score(y_test, prediction4)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred4, squared=False)}")

Recall Score: 0.0
Accuracy Score: 0.8028268551236749
Precision Score: 0.0
F1 Score: 0.0
RMSE of Probability Score: 0.4032105807452512


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



##Model 5: SVC

In [45]:
from sklearn.svm import SVC
SVC5 = SVC(probability=True).fit(X_train, y_train)

In [46]:
prob_predictions5 = SVC5.predict_proba(X_test)
pred5 = prob_predictions5[:,1]

In [47]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="SVC5 Predicted",
    mode="markers+lines", x=X_test.index, y=pred5
))

fig1.show()

In [48]:
prediction5 = SVC5.predict(X_test)

In [49]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction5)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction5)}")
print(f"Precision Score: {precision_score(y_test, prediction5)}")
print(f"F1 Score: {f1_score(y_test, prediction5)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred5, squared=False)}")

Recall Score: 0.0
Accuracy Score: 0.8028268551236749
Precision Score: 0.0
F1 Score: 0.0
RMSE of Probability Score: 0.3979226788637792


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



##Model 6: NuSVC

In [50]:
from sklearn.svm import NuSVC
NuSVC6 = NuSVC(probability=True).fit(X_train, y_train)

In [51]:
prob_predictions6 = NuSVC6.predict_proba(X_test)
pred6 = prob_predictions6[:,1]

In [52]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="SVC5 Predicted",
    mode="markers+lines", x=X_test.index, y=pred6
))

fig1.show()

In [53]:
prediction6 = NuSVC6.predict(X_test)

In [54]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction6)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction6)}")
print(f"Precision Score: {precision_score(y_test, prediction6)}")
print(f"F1 Score: {f1_score(y_test, prediction6)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred6, squared=False)}")

Recall Score: 0.0
Accuracy Score: 0.8028268551236749
Precision Score: 0.0
F1 Score: 0.0
RMSE of Probability Score: 0.3977188727029229


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



##Model 7: Linear SVC

In [55]:
from sklearn.svm import LinearSVC
LinSVC7 = CalibratedClassifierCV(LinearSVC(random_state = 42)).fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



In [56]:
prob_predictions7 = LinSVC7.predict_proba(X_test)
pred7 = prob_predictions7[:,1]

In [57]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="LinSVC7 Predicted",
    mode="markers+lines", x=X_test.index, y=pred7
))

fig1.show()

In [58]:
prediction7 = LinSVC7.predict(X_test)

In [59]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction7)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction7)}")
print(f"Precision Score: {precision_score(y_test, prediction7)}")
print(f"F1 Score: {f1_score(y_test, prediction7)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred7, squared=False)}")

Recall Score: 0.0
Accuracy Score: 0.8028268551236749
Precision Score: 0.0
F1 Score: 0.0
RMSE of Probability Score: 0.41209537493372045


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



##Model 8: K Neighbors

In [60]:
from sklearn import neighbors
KNN8 = neighbors.KNeighborsClassifier().fit(X_train, y_train)

In [61]:
prob_predictions8 = KNN8.predict_proba(X_test)
pred8 = prob_predictions8[:,1]

In [62]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="SVR8 Predicted",
    mode="markers+lines", x=X_test.index, y=pred8
))

fig1.show()

In [63]:
prediction8 = KNN8.predict(X_test)

In [64]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction8)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction8)}")
print(f"Precision Score: {precision_score(y_test, prediction8)}")
print(f"F1 Score: {f1_score(y_test, prediction8)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred8, squared=False)}")

Recall Score: 0.053763440860215055
Accuracy Score: 0.7674911660777385
Precision Score: 0.1875
F1 Score: 0.08356545961002786
RMSE of Probability Score: 0.4481607485457673


##Model 9: Gaussian Process Classification

In [65]:
#from sklearn.gaussian_process.kernels import ConstantKernel, RBF
#from sklearn.gaussian_process import GaussianProcessClassifier

#kernel = ConstantKernel(constant_value=1.0, constant_value_bounds=(0.0, 10.0)) * RBF(length_scale=0.5, length_scale_bounds=(0.0, 10.0)) + RBF(length_scale=2.0, length_scale_bounds=(0.0, 10.0))
#kernel = ConstantKernel() * RBF(length_scale=0.5, length_scale_bounds=(0.0, 10.0))
#GPC9 = GaussianProcessClassifier(kernel=kernel, random_state=42).fit(X_train, y_train)

In [66]:
#prob_predictions9 = GPC9.predict_proba(X_test)
#pred9 = prob_predictions9[:,1]

In [67]:
#fig1 = go.Figure()

#fig1.add_trace(go.Scatter(
#    name="Raw Data",
#    mode="markers+lines", x=X_test.index, y=y_test
#))
#fig1.add_trace(go.Scatter(
#    name="GPC9 Predicted",
#    mode="markers+lines", x=X_test.index, y=pred9
#))

#fig1.show()

In [68]:
#prediction9 = GPC9.predict(X_test)

In [69]:
#Number Error Metrics

#print(f"Recall Score: {recall_score(y_test, prediction9)}")
#print(f"Accuracy Score: {accuracy_score(y_test, prediction9)}")
#print(f"Precision Score: {precision_score(y_test, prediction9)}")
#print(f"F1 Score: {f1_score(y_test, prediction9)}")

#Testing Probability Measure as EWS

#print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred9, squared=False)}")

#Model 10: Naive Bayes

In [70]:
from sklearn.naive_bayes import GaussianNB
GNB10 = GaussianNB().fit(X_train, y_train)

In [71]:
prob_predictions10 = GNB10.predict_proba(X_test)
pred10 = prob_predictions10[:,1]

In [72]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="GPC9 Predicted",
    mode="markers+lines", x=X_test.index, y=pred10
))

fig1.show()

In [73]:
prediction10 = GNB10.predict(X_test)

In [74]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction10)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction10)}")
print(f"Precision Score: {precision_score(y_test, prediction10)}")
print(f"F1 Score: {f1_score(y_test, prediction10)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred10, squared=False)}")

Recall Score: 0.7634408602150538
Accuracy Score: 0.3809187279151943
Precision Score: 0.20821114369501467
F1 Score: 0.32718894009216587
RMSE of Probability Score: 0.7861787076674684


#Model 11: Decision Tree Classifier

In [75]:
from sklearn.tree import DecisionTreeClassifier
DTC11 = DecisionTreeClassifier(random_state=42).fit(X_train, y_train)

In [76]:
prob_predictions11 = DTC11.predict_proba(X_test)
pred11 = prob_predictions11[:,1]

In [77]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="DTC11 Predicted",
    mode="markers+lines", x=X_test.index, y=pred11
))

fig1.show()

In [78]:
prediction11 = DTC11.predict(X_test)

In [79]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction11)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction11)}")
print(f"Precision Score: {precision_score(y_test, prediction11)}")
print(f"F1 Score: {f1_score(y_test, prediction11)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred11, squared=False)}")

Recall Score: 0.4265232974910394
Accuracy Score: 0.6678445229681979
Precision Score: 0.2773892773892774
F1 Score: 0.3361581920903955
RMSE of Probability Score: 0.5763293130075913


##Model 12: Baggin KN Classifier

In [80]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
BAGGING12 = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5).fit(X_train, y_train)

In [81]:
prob_predictions12 = BAGGING12.predict_proba(X_test)
pred12 = prob_predictions12[:,1]

In [82]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="BAGGING12 Predicted",
    mode="markers+lines", x=X_test.index, y=pred12
))

fig1.show()

In [83]:
prediction12 = BAGGING12.predict(X_test)

In [84]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction11)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction11)}")
print(f"Precision Score: {precision_score(y_test, prediction11)}")
print(f"F1 Score: {f1_score(y_test, prediction11)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred11, squared=False)}")

Recall Score: 0.4265232974910394
Accuracy Score: 0.6678445229681979
Precision Score: 0.2773892773892774
F1 Score: 0.3361581920903955
RMSE of Probability Score: 0.5763293130075913


##Model 13: Random Forest Classifier

In [85]:
from sklearn.ensemble import RandomForestClassifier
RF13 = RandomForestClassifier(n_estimators=1000).fit(X_train, y_train)

In [86]:
prob_predictions13 = RF13.predict_proba(X_test)
pred13 = prob_predictions13[:,1]

In [87]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="RF13 Predicted",
    mode="markers+lines", x=X_test.index, y=pred13
))

fig1.show()

In [88]:
prediction13 = RF13.predict(X_test)

In [89]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction13)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction13)}")
print(f"Precision Score: {precision_score(y_test, prediction13)}")
print(f"F1 Score: {f1_score(y_test, prediction13)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred13, squared=False)}")

Recall Score: 0.17204301075268819
Accuracy Score: 0.7003533568904594
Precision Score: 0.1991701244813278
F1 Score: 0.1846153846153846
RMSE of Probability Score: 0.46437070910753164


##Model 14: Extreme Randomized Trees

In [90]:
from sklearn.ensemble import ExtraTreesClassifier
ETC14 = ExtraTreesClassifier(random_state=42).fit(X_train, y_train)

In [91]:
prob_predictions14 = ETC14.predict_proba(X_test)
pred14 = prob_predictions14[:,1]

In [92]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="ETC14 Predicted",
    mode="markers+lines", x=X_test.index, y=pred14
))

fig1.show()

In [93]:
prediction14 = ETC14.predict(X_test)

In [94]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction14)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction14)}")
print(f"Precision Score: {precision_score(y_test, prediction14)}")
print(f"F1 Score: {f1_score(y_test, prediction14)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred14, squared=False)}")

Recall Score: 0.1003584229390681
Accuracy Score: 0.7533568904593639
Precision Score: 0.2222222222222222
F1 Score: 0.1382716049382716
RMSE of Probability Score: 0.4541804252246499


#Model 15: AdaBoost CLassifier

In [95]:
from sklearn.ensemble import AdaBoostClassifier
AB15 = AdaBoostClassifier(n_estimators=100).fit(X_train, y_train)

In [96]:
prob_predictions15 = AB15.predict_proba(X_test)
pred15 = prob_predictions15[:,1]

In [97]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="AB15 Predicted",
    mode="markers+lines", x=X_test.index, y=pred15
))

fig1.show()

In [98]:
prediction15 = AB15.predict(X_test)

In [99]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction15)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction15)}")
print(f"Precision Score: {precision_score(y_test, prediction15)}")
print(f"F1 Score: {f1_score(y_test, prediction15)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred15, squared=False)}")

Recall Score: 0.3835125448028674
Accuracy Score: 0.7795053003533569
Precision Score: 0.4331983805668016
F1 Score: 0.4068441064638783
RMSE of Probability Score: 0.4949480162633425


##Mode 16: Gradient Tree Boosting

In [100]:
from sklearn.ensemble import GradientBoostingClassifier
GTB16 = GradientBoostingClassifier(random_state = 42).fit(X_train, y_train)

In [101]:
prob_predictions16 = GTB16.predict_proba(X_test)
pred16 = prob_predictions16[:,1]

In [102]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="GTB16 Predicted",
    mode="markers+lines", x=X_test.index, y=pred16
))

fig1.show()

In [103]:
prediction16 = GTB16.predict(X_test)

In [104]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction16)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction16)}")
print(f"Precision Score: {precision_score(y_test, prediction16)}")
print(f"F1 Score: {f1_score(y_test, prediction16)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred16, squared=False)}")

Recall Score: 0.5591397849462365
Accuracy Score: 0.7455830388692579
Precision Score: 0.3969465648854962
F1 Score: 0.46428571428571425
RMSE of Probability Score: 0.42593342647448623


##Model 17: Voting CLassifier

In [105]:
from sklearn.ensemble import VotingClassifier

clf1 = LogisticRegression(random_state=1, max_iter=5000)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)], voting='hard')

for clf, label in zip([clf1, clf2, clf3, eclf], ['Logistic Regression', 'Random Forest', 'naive Bayes', 'Ensemble']):
  scores = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=5)
  print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.52 (+/- 0.21) [Logistic Regression]
Accuracy: 0.53 (+/- 0.19) [Random Forest]
Accuracy: 0.41 (+/- 0.19) [naive Bayes]
Accuracy: 0.52 (+/- 0.18) [Ensemble]


##Model 18: XGBoost

In [106]:
XGB18 = XGBClassifier(random_state=42).fit(X_train, y_train)

In [107]:
prob_predictions18 = XGB18.predict_proba(X_test)
pred18 = prob_predictions18[:,1]

In [108]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="XGB18 Predicted",
    mode="markers+lines", x=X_test.index, y=pred18
))

fig1.show()

In [109]:
prediction18 = XGB18.predict(X_test)

In [110]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction18)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction18)}")
print(f"Precision Score: {precision_score(y_test, prediction18)}")
print(f"F1 Score: {f1_score(y_test, prediction18)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred18, squared=False)}")

Recall Score: 0.46953405017921146
Accuracy Score: 0.8353356890459364
Precision Score: 0.6064814814814815
F1 Score: 0.5292929292929293
RMSE of Probability Score: 0.3570421805321198


#Model 19: MLP Classification

In [111]:
from sklearn.neural_network import MLPClassifier
MLP19 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=42).fit(X_train, y_train)

In [112]:
prob_predictions19 = MLP19.predict_proba(X_test)
pred19 = prob_predictions19[:,1]

In [113]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="MLP19 Predicted",
    mode="markers+lines", x=X_test.index, y=pred19
))

fig1.show()

In [114]:
prediction19 = MLP19.predict(X_test)

In [115]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction19)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction19)}")
print(f"Precision Score: {precision_score(y_test, prediction19)}")
print(f"F1 Score: {f1_score(y_test, prediction19)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred19, squared=False)}")

Recall Score: 0.0
Accuracy Score: 0.8028268551236749
Precision Score: 0.0
F1 Score: 0.0
RMSE of Probability Score: 0.40309865402184886


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



##Model 20: MLPClassification Large

In [116]:
MLP20 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(10, 10), random_state=42).fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html



In [117]:
prob_predictions20 = MLP20.predict_proba(X_test)
pred20 = prob_predictions20[:,1]

In [118]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    name="Raw Data",
    mode="markers+lines", x=X_test.index, y=y_test
))
fig1.add_trace(go.Scatter(
    name="MLP20 Predicted",
    mode="markers+lines", x=X_test.index, y=pred20
))

fig1.show()

In [119]:
prediction20 = MLP20.predict(X_test)

In [120]:
#Number Error Metrics

print(f"Recall Score: {recall_score(y_test, prediction20)}")
print(f"Accuracy Score: {accuracy_score(y_test, prediction20)}")
print(f"Precision Score: {precision_score(y_test, prediction20)}")
print(f"F1 Score: {f1_score(y_test, prediction20)}")

#Testing Probability Measure as EWS

print(f"RMSE of Probability Score: {mean_squared_error(y_test, pred20, squared=False)}")

Recall Score: 0.007168458781362007
Accuracy Score: 0.8014134275618374
Precision Score: 0.3333333333333333
F1 Score: 0.014035087719298246
RMSE of Probability Score: 0.40742126761165115


#Validating EWS

In [121]:
prob_predictions_EWS = XGB18.predict_proba(X)
pred_EWS = prob_predictions_EWS[:,1]

In [122]:
df_crisis = pd.DataFrame(pred_EWS, columns=['CrisisProb'])

In [123]:
df_crisis.index = X.index

In [124]:
df_crisis

,CrisisProb
Date,
2004-01-23,0.042746
2004-01-26,0.082748
2004-01-27,0.054984
2004-01-28,0.040246
2004-01-29,0.043084
...,...
2021-09-09,0.131596
2021-09-10,0.820952
2021-09-11,0.219361


In [125]:
fig1 = go.Figure()

#fig1.add_trace(go.Scatter(
#    name="Raw Data",
#    mode="markers+lines", x=X, y=y
#))
fig1.add_trace(go.Scatter(
    name="MLP20 Predicted",
    mode="markers+lines", x=X.index, y=pred_EWS
))

fig1.show()

In [126]:
#Import VIX Data

df_vix = pd.read_csv('/content/VIX Cleaned Data.csv')

In [127]:
df_vix['Date'] = pd.to_datetime(df_vix.iloc[:, 0], infer_datetime_format=True)

In [128]:
reg_df = X.merge(df_vix, how = 'inner', on = ['Date'])

In [129]:
X

,CHF/EUR Log Difference_lag_1,CHF/EUR Log Difference_lag_2,CHF/EUR Log Difference_lag_3,CHF/EUR Log Difference_lag_4,CHF/EUR Log Difference_lag_5,CHF/EUR Log Difference_lag_6,CHF/EUR Log Difference_lag_7,CHF/EUR Log Difference_lag_8,CHF/EUR Log Difference_lag_9,CHF/EUR Log Difference_lag_10,CHF/EUR Log Difference_lag_11,CHF/EUR Log Difference_lag_12,CHF/EUR Log Difference_lag_13,swiss franc_lag_1,swiss franc_lag_2,swiss franc_lag_3,swiss franc_lag_4,swiss franc_lag_5,swiss franc_lag_6,swiss franc_lag_7,swiss franc_lag_8,swiss franc_lag_9,swiss franc_lag_10,swiss franc_lag_11,swiss franc_lag_12,swiss franc_lag_13,Europe_lag_1,Europe_lag_2,Europe_lag_3,Europe_lag_4,Europe_lag_5,Europe_lag_6,Europe_lag_7,Europe_lag_8,Europe_lag_9,Europe_lag_10,Europe_lag_11,Europe_lag_12,Europe_lag_13,EUR_lag_1,...,bubble_lag_13,energy_lag_1,energy_lag_2,energy_lag_3,energy_lag_4,energy_lag_5,energy_lag_6,energy_lag_7,energy_lag_8,energy_lag_9,energy_lag_10,energy_lag_11,energy_lag_12,energy_lag_13,1939_lag_1,1939_lag_2,1939_lag_3,1939_lag_4,1939_lag_5,1939_lag_6,1939_lag_7,1939_lag_8,1939_lag_9,1939_lag_10,1939_lag_11,1939_lag_12,1939_lag_13,CHF/EUR_RealVol_lag_1,CHF/EUR_RealVol_lag_2,CHF/EUR_RealVol_lag_3,CHF/EUR_RealVol_lag_4,CHF/EUR_RealVol_lag_5,CHF/EUR_RealVol_lag_6,CHF/EUR_RealVol_lag_7,CHF/EUR_RealVol_lag_8,CHF/EUR_RealVol_lag_9,CHF/EUR_RealVol_lag_10,CHF/EUR_RealVol_lag_11,CHF/EUR_RealVol_lag_12,CHF/EUR_RealVol_lag_13
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2004-01-23,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,-0.002875,-0.002498,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,73.0,39.0,8.0,...,23.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,84.0,52.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,45.0,15.0,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0.004228,0.038119
2004-01-26,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,-0.002875,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,73.0,6.0,...,29.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,84.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,45.0,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0.004228
2004-01-27,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,5.0,...,31.0,73.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987
2004-01-28,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,12.0,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,79.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,2.0,...,30.0,89.0,73.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,9.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,0.019360,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333
2004-01-29,0.000383,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.0,12.0,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,72.0,79.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,7.0,...,26.0,78.0,89.0,73.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,21.0,9.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,0.005731,0.019360,0.042302,0.037996,0.04585

In [130]:
reg_df['CHF/EUR_RealVol'] = reg_df['CHF/EUR_RealVol_lag_1'].shift(-1)
reg_df.dropna()

,Date,CHF/EUR Log Difference_lag_1,CHF/EUR Log Difference_lag_2,CHF/EUR Log Difference_lag_3,CHF/EUR Log Difference_lag_4,CHF/EUR Log Difference_lag_5,CHF/EUR Log Difference_lag_6,CHF/EUR Log Difference_lag_7,CHF/EUR Log Difference_lag_8,CHF/EUR Log Difference_lag_9,CHF/EUR Log Difference_lag_10,CHF/EUR Log Difference_lag_11,CHF/EUR Log Difference_lag_12,CHF/EUR Log Difference_lag_13,swiss franc_lag_1,swiss franc_lag_2,swiss franc_lag_3,swiss franc_lag_4,swiss franc_lag_5,swiss franc_lag_6,swiss franc_lag_7,swiss franc_lag_8,swiss franc_lag_9,swiss franc_lag_10,swiss franc_lag_11,swiss franc_lag_12,swiss franc_lag_13,Europe_lag_1,Europe_lag_2,Europe_lag_3,Europe_lag_4,Europe_lag_5,Europe_lag_6,Europe_lag_7,Europe_lag_8,Europe_lag_9,Europe_lag_10,Europe_lag_11,Europe_lag_12,Europe_lag_13,...,energy_lag_3,energy_lag_4,energy_lag_5,energy_lag_6,energy_lag_7,energy_lag_8,energy_lag_9,energy_lag_10,energy_lag_11,energy_lag_12,energy_lag_13,1939_lag_1,1939_lag_2,1939_lag_3,1939_lag_4,1939_lag_5,1939_lag_6,1939_lag_7,1939_lag_8,1939_lag_9,1939_lag_10,1939_lag_11,1939_lag_12,1939_lag_13,CHF/EUR_RealVol_lag_1,CHF/EUR_RealVol_lag_2,CHF/EUR_RealVol_lag_3,CHF/EUR_RealVol_lag_4,CHF/EUR_RealVol_lag_5,CHF/EUR_RealVol_lag_6,CHF/EUR_RealVol_lag_7,CHF/EUR_RealVol_lag_8,CHF/EUR_RealVol_lag_9,CHF/EUR_RealVol_lag_10,CHF/EUR_RealVol_lag_11,CHF/EUR_RealVol_lag_12,CHF/EUR_RealVol_lag_13,Adj Close,Log Return,CHF/EUR_RealVol
0,2004-01-23,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,-0.002875,-0.002498,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,73.0,39.0,...,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,84.0,52.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,45.0,15.0,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0.004228,0.038119,14.840000,0.879870,0.037996
1,2004-01-26,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,-0.002875,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,73.0,...,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,84.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,45.0,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0.004228,14.550000,-1.973524,0.042302
2,2004-01-27,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,...,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,15.350000,5.352448,0.019360
3,2004-01-28,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,12.0,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,79.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,...,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,9.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,0.019360,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,16.780001,8.907229,0.005731
4,2004-01-29,0.000383,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.0,12.0,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,72.0,79.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,...,73.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,21.0,9.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,0.005731,0.019360,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0

In [131]:
pred_df = df_crisis.merge(reg_df, how = 'inner', on = ['Date'])

In [132]:
pred_df = pred_df.dropna()

In [133]:
pred_df.head(5)

,Date,CrisisProb,CHF/EUR Log Difference_lag_1,CHF/EUR Log Difference_lag_2,CHF/EUR Log Difference_lag_3,CHF/EUR Log Difference_lag_4,CHF/EUR Log Difference_lag_5,CHF/EUR Log Difference_lag_6,CHF/EUR Log Difference_lag_7,CHF/EUR Log Difference_lag_8,CHF/EUR Log Difference_lag_9,CHF/EUR Log Difference_lag_10,CHF/EUR Log Difference_lag_11,CHF/EUR Log Difference_lag_12,CHF/EUR Log Difference_lag_13,swiss franc_lag_1,swiss franc_lag_2,swiss franc_lag_3,swiss franc_lag_4,swiss franc_lag_5,swiss franc_lag_6,swiss franc_lag_7,swiss franc_lag_8,swiss franc_lag_9,swiss franc_lag_10,swiss franc_lag_11,swiss franc_lag_12,swiss franc_lag_13,Europe_lag_1,Europe_lag_2,Europe_lag_3,Europe_lag_4,Europe_lag_5,Europe_lag_6,Europe_lag_7,Europe_lag_8,Europe_lag_9,Europe_lag_10,Europe_lag_11,Europe_lag_12,...,energy_lag_3,energy_lag_4,energy_lag_5,energy_lag_6,energy_lag_7,energy_lag_8,energy_lag_9,energy_lag_10,energy_lag_11,energy_lag_12,energy_lag_13,1939_lag_1,1939_lag_2,1939_lag_3,1939_lag_4,1939_lag_5,1939_lag_6,1939_lag_7,1939_lag_8,1939_lag_9,1939_lag_10,1939_lag_11,1939_lag_12,1939_lag_13,CHF/EUR_RealVol_lag_1,CHF/EUR_RealVol_lag_2,CHF/EUR_RealVol_lag_3,CHF/EUR_RealVol_lag_4,CHF/EUR_RealVol_lag_5,CHF/EUR_RealVol_lag_6,CHF/EUR_RealVol_lag_7,CHF/EUR_RealVol_lag_8,CHF/EUR_RealVol_lag_9,CHF/EUR_RealVol_lag_10,CHF/EUR_RealVol_lag_11,CHF/EUR_RealVol_lag_12,CHF/EUR_RealVol_lag_13,Adj Close,Log Return,CHF/EUR_RealVol
0,2004-01-23,0.042746,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,-0.002875,-0.002498,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,73.0,...,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,84.0,52.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,45.0,15.0,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0.004228,0.038119,14.840000,0.879870,0.037996
1,2004-01-26,0.082748,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,-0.002875,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,...,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,84.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,45.0,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0.004228,14.550000,-1.973524,0.042302
2,2004-01-27,0.054984,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,...,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,15.350000,5.352448,0.019360
3,2004-01-28,0.040246,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,12.0,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,79.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,...,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,9.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,0.019360,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,16.780001,8.907229,0.005731
4,2004-01-29,0.043084,0.000383,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.0,12.0,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,72.0,79.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,...,73.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,21.0,9.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,0.005731,0.019360,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.

In [134]:
endo = pred_df[['CHF/EUR_RealVol']]
exo = sm.add_constant(pred_df[['Log Return', 'CrisisProb']])

In [135]:
mod = sm.OLS(endo, exo)
res = mod.fit()

In [136]:
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:        CHF/EUR_RealVol   R-squared:                       0.258
Model:                            OLS   Adj. R-squared:                  0.258
Method:                 Least Squares   F-statistic:                     773.2
Date:                Mon, 22 Nov 2021   Prob (F-statistic):          8.37e-289
Time:                        20:25:58   Log-Likelihood:                 6432.5
No. Observations:                4439   AIC:                        -1.286e+04
Df Residuals:                    4436   BIC:                        -1.284e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0071      0.001      5.920      0.0

In [137]:
endo = pred_df[['CHF/EUR_RealVol']]
exo = sm.add_constant(pred_df[['Log Return']])
mod = sm.OLS(endo, exo)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:        CHF/EUR_RealVol   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.8530
Date:                Mon, 22 Nov 2021   Prob (F-statistic):              0.356
Time:                        20:25:58   Log-Likelihood:                 5769.2
No. Observations:                4439   AIC:                        -1.153e+04
Df Residuals:                    4437   BIC:                        -1.152e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0400      0.001     40.365      0.0

In [138]:
endo = pred_df[['CHF/EUR_RealVol']]
exo = sm.add_constant(pred_df[['CrisisProb']])
mod = sm.OLS(endo, exo)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:        CHF/EUR_RealVol   R-squared:                       0.258
Model:                            OLS   Adj. R-squared:                  0.258
Method:                 Least Squares   F-statistic:                     1544.
Date:                Mon, 22 Nov 2021   Prob (F-statistic):          5.05e-290
Time:                        20:25:58   Log-Likelihood:                 6431.4
No. Observations:                4439   AIC:                        -1.286e+04
Df Residuals:                    4437   BIC:                        -1.285e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0071      0.001      5.931      0.0

#Rolling T-Test Validation

In [147]:
index_dates = ts_df.iloc[100:len(ts_df)-100].index

In [149]:
from scipy import stats

rolling_statistic = []
rolling_p_value = []

for i in range(100, len(ts_df), 1):
  sub = pred_df.iloc[i-99:i,:]
  sub_t_test = stats.ttest_ind(sub['CHF/EUR_RealVol'], sub['CrisisProb'])
  rolling_statistic.append(sub_t_test[0])
  rolling_p_value.append(sub_t_test[1])

fig1 = go.Figure([go.Scatter(name = 'CHF/EUR', x=index_dates, y=rolling_p_value)])

fig1.update_layout(
    xaxis_title="Date",
    yaxis_title="p-value",
    width=2000,
    height=1000,
    font = dict(
        size=25
    )
)

fig1.add_shape(
        type='line',
        x0=index_dates.min(),
        y0=0.05,
        x1=index_dates.max(),
        y1=0.05,
        line=dict(
            color='Red',
        )
)

fig1.show()

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:3622: RuntimeWarning:

Degrees of freedom <= 0 for slice

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning:

invalid value encountered in double_scalars

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:226: RuntimeWarning:

invalid value encountered in true_divide



#Validating EWS Tree Boost

In [150]:
prob_predictionsEWS = GTB16.predict_proba(X)
predEWS = prob_predictionsEWS[:,1]

In [151]:
df_crisis = pd.DataFrame(predEWS, columns=['CrisisProb'])
df_crisis.index = X.index

In [152]:
reg_df = X.merge(df_vix, how = 'inner', on = ['Date'])

In [153]:
reg_df['CHF/EUR_RealVol'] = reg_df['CHF/EUR_RealVol_lag_1'].shift(-1)
reg_df.dropna()

,Date,CHF/EUR Log Difference_lag_1,CHF/EUR Log Difference_lag_2,CHF/EUR Log Difference_lag_3,CHF/EUR Log Difference_lag_4,CHF/EUR Log Difference_lag_5,CHF/EUR Log Difference_lag_6,CHF/EUR Log Difference_lag_7,CHF/EUR Log Difference_lag_8,CHF/EUR Log Difference_lag_9,CHF/EUR Log Difference_lag_10,CHF/EUR Log Difference_lag_11,CHF/EUR Log Difference_lag_12,CHF/EUR Log Difference_lag_13,swiss franc_lag_1,swiss franc_lag_2,swiss franc_lag_3,swiss franc_lag_4,swiss franc_lag_5,swiss franc_lag_6,swiss franc_lag_7,swiss franc_lag_8,swiss franc_lag_9,swiss franc_lag_10,swiss franc_lag_11,swiss franc_lag_12,swiss franc_lag_13,Europe_lag_1,Europe_lag_2,Europe_lag_3,Europe_lag_4,Europe_lag_5,Europe_lag_6,Europe_lag_7,Europe_lag_8,Europe_lag_9,Europe_lag_10,Europe_lag_11,Europe_lag_12,Europe_lag_13,...,energy_lag_3,energy_lag_4,energy_lag_5,energy_lag_6,energy_lag_7,energy_lag_8,energy_lag_9,energy_lag_10,energy_lag_11,energy_lag_12,energy_lag_13,1939_lag_1,1939_lag_2,1939_lag_3,1939_lag_4,1939_lag_5,1939_lag_6,1939_lag_7,1939_lag_8,1939_lag_9,1939_lag_10,1939_lag_11,1939_lag_12,1939_lag_13,CHF/EUR_RealVol_lag_1,CHF/EUR_RealVol_lag_2,CHF/EUR_RealVol_lag_3,CHF/EUR_RealVol_lag_4,CHF/EUR_RealVol_lag_5,CHF/EUR_RealVol_lag_6,CHF/EUR_RealVol_lag_7,CHF/EUR_RealVol_lag_8,CHF/EUR_RealVol_lag_9,CHF/EUR_RealVol_lag_10,CHF/EUR_RealVol_lag_11,CHF/EUR_RealVol_lag_12,CHF/EUR_RealVol_lag_13,Adj Close,Log Return,CHF/EUR_RealVol
0,2004-01-23,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,-0.002875,-0.002498,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,73.0,39.0,...,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,84.0,52.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,45.0,15.0,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0.004228,0.038119,14.840000,0.879870,0.037996
1,2004-01-26,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,-0.002875,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,73.0,...,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,84.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,45.0,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,0.004228,14.550000,-1.973524,0.042302
2,2004-01-27,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,0.000064,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,60.0,...,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,87.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,20.0,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,0.032987,15.350000,5.352448,0.019360
3,2004-01-28,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.001341,12.0,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0,79.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,67.0,...,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,80.0,9.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,17.0,0.019360,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0.025801,0.014333,16.780001,8.907229,0.005731
4,2004-01-29,0.000383,-0.000128,-0.001852,0.001916,-0.001469,0.002617,-0.000957,-0.000574,-0.004671,0.000321,0.002881,0.001661,-0.000958,0.0,12.0,0.0,14.0,0.0,0.0,24.0,0.0,12.0,0.0,0.0,0.0,0.0,72.0,79.0,79.0,70.0,74.0,66.0,80.0,61.0,64.0,69.0,66.0,68.0,75.0,...,73.0,70.0,74.0,88.0,74.0,76.0,61.0,82.0,83.0,94.0,67.0,21.0,9.0,13.0,24.0,33.0,6.0,3.0,14.0,20.0,19.0,15.0,23.0,7.0,0.005731,0.019360,0.042302,0.037996,0.045859,0.040110,0.004290,0.045984,0.056030,0.028740,0.013697,0.029394,0

In [154]:
pred_df = df_crisis.merge(reg_df, how = 'inner', on = ['Date'])
pred_df = pred_df.dropna()

In [155]:
endo = pred_df[['CHF/EUR_RealVol']]
exo = sm.add_constant(pred_df[['CrisisProb']])
mod = sm.OLS(endo, exo)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:        CHF/EUR_RealVol   R-squared:                       0.209
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     1170.
Date:                Mon, 22 Nov 2021   Prob (F-statistic):          8.05e-228
Time:                        20:56:13   Log-Likelihood:                 6288.3
No. Observations:                4439   AIC:                        -1.257e+04
Df Residuals:                    4437   BIC:                        -1.256e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0094      0.001      7.529      0.0